In [9]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 

In [10]:
data = pd.read_csv("data/FinalBalancedDataset.csv")

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56745 entries, 0 to 56744
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  56745 non-null  int64 
 1   Toxicity    56745 non-null  int64 
 2   tweet       56745 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [12]:
data.head(5)

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


In [13]:
# Dropping the unesscessary heading -
data = data.drop("Unnamed: 0",axis=1)

In [14]:
data.head(5)

,Toxicity,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [15]:
data['Toxicity'].value_counts() # 1- toxic and 0-non-toxic

Toxicity
0    32592
1    24153
Name: count, dtype: int64

In [16]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import WordNetLemmatizer
from nltk import pos_tag,word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
import re

### Text pre-processing -

In [4]:
wordnet_lemmatizer = WordNetLemmatizer()

In [5]:
def prepare_text(text):
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = text.split()
    text = ' '.join(text)
    text = word_tokenize(text)
    text = pos_tag(text)
    lemma = []
    for i in text: lemma.append(wordnet_lemmatizer.lemmatize(i[0], pos = get_wordnet_pos(i[1])))
    lemma = ' '.join(lemma)
    return lemma

In [19]:
data['clean_tweets'] = data['tweet'].apply(lambda x:prepare_text(x))

In [20]:
data.head(5)


,Toxicity,tweet,clean_tweets
0,0,@user when a father is dysfunctional and is s...,user when a father be dysfunctional and be so ...
1,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i ca n't use ...
2,0,bihday your majesty,bihday your majesty
3,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,0,factsguide: society now #motivation,factsguide society now motivation


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')


In [22]:
corpus = data['clean_tweets'].values.astype('U')

In [27]:
stopwords = set(nltk_stopwords.words('english'))

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_list = list(stopwords)
count_tf_idf = TfidfVectorizer(stop_words=stopwords_list)
tf_idf = count_tf_idf.fit_transform(corpus)

     

In [34]:
import pickle # serializable library 

In [35]:
pickle.dump(count_tf_idf,open("tf_idf.pkt","wb")) 

In [36]:
tf_idf_train, tf_idf_test, target_train, target_test = train_test_split(
    tf_idf, data['Toxicity'], test_size = 0.8, random_state= 42, shuffle=True
)

### Creating a Binary Classification Model --

In [37]:
model_bayes = MultinomialNB()
model_bayes = model_bayes.fit(tf_idf_train,target_train)

In [38]:
y_pred_proba = model_bayes.predict_proba(tf_idf_test)[::, 1]
     

In [39]:
y_pred_proba

array([0.90152453, 0.27916787, 0.79021827, ..., 0.09487729, 0.20555162,
       0.32090192])

In [40]:
fpr, tpr, _ = roc_curve(target_test, y_pred_proba)

In [41]:
final_roc_auc = roc_auc_score(target_test, y_pred_proba)

In [42]:
final_roc_auc

0.9658691315317345

In [43]:
test_text = "I hate you moron" # Toxic Comment
test_tfidf = count_tf_idf.transform([test_text])
display(model_bayes.predict_proba(test_tfidf))
display(model_bayes.predict(test_tfidf))

array([[0.39920068, 0.60079932]])

array([1], dtype=int64)

In [44]:
test_text = "It was an amazing experince" # Non - Toxic -Non toxic comment values are represented by 0 and toxic one in 1.
test_tfidf = count_tf_idf.transform([test_text])
display(model_bayes.predict_proba(test_tfidf))
display(model_bayes.predict(test_tfidf))

array([[0.9583188, 0.0416812]])

array([0], dtype=int64)

### Save the model

In [45]:
pickle.dump(model_bayes, open("toxicity_model.pkt", "wb"))